# Visualización de Embeddings y Documentos en ChromaDB

Este notebook muestra cómo visualizar los embeddings y documentos almacenados en la base de datos vectorial Chroma que hemos creado. Utilizaremos Plotly para crear gráficas interactivas.

In [7]:
# Importamos las bibliotecas necesarias
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

## Conexión a ChromaDB

Primero conectamos a la base de datos Chroma existente:

In [8]:
# Configuramos el modelo de embeddings
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

# Conectamos a la base de datos
db_location = "./chroma_langchain_db"
vector_store = Chroma(
    collection_name="AMVA-Reviews",
    persist_directory=db_location,
    embedding_function=embeddings
)

print(f"Conectado a la colección: {vector_store._collection.name}")

Conectado a la colección: AMVA-Reviews


## Recuperación de Documentos

Vamos a recuperar todos los documentos almacenados en la base de datos:

In [9]:
# Obtener todos los documentos
results = vector_store.get()

# Crear un dataframe para facilitar la visualización
documents_df = pd.DataFrame({
    'id': results['ids'],
    'document': results['documents'],
    'metadata': results['metadatas']
})

# Extraer los tipos de ubicación
location_types = []
for metadata in results['metadatas']:
    location_types.append(metadata['Location-Type'])

documents_df['location_type'] = location_types

print(f"Total de documentos: {len(documents_df)}")
documents_df

Total de documentos: 20


,id,document,metadata,location_type
0,0,Belén 8.83 217501 Arepa de Choclo Parque Bibli...,{'Location-Type': 'Commune'},Commune
1,1,La Candelaria 3.5 100000 Bandeja Paisa Plaza B...,{'Location-Type': 'Commune'},Commune
2,2,El Poblado 23.1 130000 Sancocho Antioqueño Par...,{'Location-Type': 'Commune'},Commune
3,3,Robledo 13.5 150000 Empanadas Cerro El Volador...,{'Location-Type': 'Commune'},Commune
4,4,Manrique 7.5 120000 Chorizo Antioqueño Parque ...,{'Location-Type': 'Commune'},Commune
5,5,Villa Hermosa 5.8 104450 Arepa de Queso Parque...,{'Location-Type': 'Commune'},Commune
6,6,Guayabal 7.6 76355 Lechona Parque Juan Pablo I...,{'Location-Type': 'Commune'},Commune
7,7,Castilla 6.2 110000 Frijoles con Chicharrón Pa...,{'Location-Type': 'Commune'},Commune
8,8,Santa Cruz 5.5 95000 Chicharrón Parque Santa C...,{'Location-Type': 'Commune'},Commune
9,9,Doce de Octubre 6.0 98000 Mondongo Parque Doce...,{'Location-Type': 'Commune'},Commune


## Visualización de Documentos por Tipo

Visualicemos la distribución de documentos por tipo de ubicación:

In [10]:
# Contar los tipos de ubicaciones
location_counts = pd.Series(location_types).value_counts().reset_index()
location_counts.columns = ['Tipo', 'Cantidad']

# Crear gráfico de barras con Plotly
fig = px.bar(
    location_counts, 
    x='Tipo', 
    y='Cantidad',
    color='Tipo',
    title='Distribución de Documentos por Tipo de Ubicación',
    labels={'Tipo': 'Tipo de Ubicación', 'Cantidad': 'Número de Documentos'},
    template='plotly_white'
)

fig.update_layout(xaxis_title='Tipo de Ubicación', yaxis_title='Número de Documentos')
fig.show()

## Obtención y Visualización de Embeddings

Ahora vamos a obtener los embeddings de los documentos. Si Chroma no los devuelve directamente, los generaremos manualmente:

In [11]:
# Intentar acceder a los embeddings directamente desde Chroma
embeddings_data = results.get('embeddings')

# Si los embeddings no están disponibles, generarlos manualmente
if not embeddings_data:
    print("Generando embeddings manualmente...")
    embeddings_data = []
    
    for doc in results['documents']:
        emb = embeddings.embed_query(doc)
        embeddings_data.append(emb)
    
    print(f"Embeddings generados: {len(embeddings_data)}")
else:
    print(f"Embeddings recuperados directamente: {len(embeddings_data)}")

# Convertir a array numpy para análisis
embeddings_array = np.array(embeddings_data)

print(f"Dimensiones del array de embeddings: {embeddings_array.shape}")
print(f"Media de valores: {np.mean(embeddings_array)}")
print(f"Desviación estándar: {np.std(embeddings_array)}")

# Mostrar un ejemplo de embedding
print("\nPrimeros 10 valores del primer embedding:")
print(embeddings_array[0][:10])

Generando embeddings manualmente...
Embeddings generados: 20
Dimensiones del array de embeddings: (20, 1024)
Media de valores: -0.00013099255779127155
Desviación estándar: 0.031249724073849534

Primeros 10 valores del primer embedding:
[-0.04342313  0.01965761  0.00702177  0.00271206  0.01587664  0.02088998
 -0.03535728  0.0083636   0.03835624  0.0282686 ]


## Visualización de Embeddings con Reducción de Dimensionalidad

Los embeddings son vectores de alta dimensionalidad (cientos o miles de dimensiones), por lo que necesitamos reducir su dimensionalidad para visualizarlos. Usaremos PCA y t-SNE:

In [16]:
# Primero, aplicamos normalización a los embeddings para evitar problemas numéricos
import warnings
from sklearn.preprocessing import RobustScaler
from sklearn.exceptions import ConvergenceWarning

# Suprimir advertencias específicas de sklearn
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)

# Verificar si hay valores problemáticos en los embeddings
print(f"Verificando datos: NaN: {np.isnan(embeddings_array).any()}, "
      f"Inf: {np.isinf(embeddings_array).any()}")
print(f"Valor mínimo: {np.min(embeddings_array)}, Valor máximo: {np.max(embeddings_array)}")

# Limpiar datos: reemplazar valores inf y nan si existen
embeddings_clean = np.nan_to_num(embeddings_array, nan=0.0, posinf=1.0, neginf=-1.0)

# Usar RobustScaler que es menos sensible a valores atípicos
scaler = RobustScaler()
embeddings_scaled = scaler.fit_transform(embeddings_clean)

# Verificar nuevamente después de la limpieza y escalado
print(f"Después de limpieza: NaN: {np.isnan(embeddings_scaled).any()}, "
      f"Inf: {np.isinf(embeddings_scaled).any()}")
print(f"Valor mínimo: {np.min(embeddings_scaled)}, Valor máximo: {np.max(embeddings_scaled)}")

# Aplicar PCA con manejo de errores
try:
    # Usar svd_solver='arpack' que suele ser más estable para matrices problemáticas
    pca = PCA(n_components=3, svd_solver='arpack', random_state=42)
    embeddings_3d = pca.fit_transform(embeddings_scaled)
    varianza_explicada = pca.explained_variance_ratio_ * 100
    
    print(f"Varianza explicada por cada componente:")
    print(f"Componente 1: {varianza_explicada[0]:.2f}%")
    print(f"Componente 2: {varianza_explicada[1]:.2f}%")
    print(f"Componente 3: {varianza_explicada[2]:.2f}%")
    print(f"Total: {sum(varianza_explicada):.2f}%")
    
    # Si todo funciona bien con PCA, continuamos con la visualización
    viz_df = pd.DataFrame({
        'x': embeddings_3d[:, 0],
        'y': embeddings_3d[:, 1],
        'z': embeddings_3d[:, 2],
        'documento': [doc.split()[0] for doc in results['documents']],
        'documento_completo': results['documents'],
        'tipo': location_types
    })
    
    # Crear gráfico 3D interactivo
    fig = px.scatter_3d(
        viz_df, 
        x='x', 
        y='y', 
        z='z',
        color='tipo',
        hover_data=['documento_completo'],
        text='documento',
        title=f'Visualización 3D de Embeddings (PCA - {sum(varianza_explicada):.2f}% de varianza explicada)',
        labels={
            'x': f'Componente 1 ({varianza_explicada[0]:.2f}%)', 
            'y': f'Componente 2 ({varianza_explicada[1]:.2f}%)', 
            'z': f'Componente 3 ({varianza_explicada[2]:.2f}%)'
        }
    )
    
    fig.update_traces(
        marker=dict(size=10, opacity=0.8), 
        selector=dict(mode='markers+text'),
        textposition='top center'
    )
    
    fig.update_layout(
        scene=dict(
            xaxis_title=f'Componente 1 ({varianza_explicada[0]:.2f}%)',
            yaxis_title=f'Componente 2 ({varianza_explicada[1]:.2f}%)',
            zaxis_title=f'Componente 3 ({varianza_explicada[2]:.2f}%)'
        ),
        legend_title_text='Tipo de Ubicación',
        height=800,
        width=1000,
        margin=dict(l=0, r=0, t=40, b=0)
    )
    
    fig.show()
    
except Exception as e:
    print(f"Error al aplicar PCA: {e}")
    print("Intentando con un enfoque alternativo: UMAP")
    
    # Si PCA falla, intentamos con UMAP que es más robusto
    try:
        # Importar UMAP (si no está instalado, ejecute: pip install umap-learn)
        from umap import UMAP
        
        # UMAP es más robusto para embeddings
        reducer = UMAP(n_components=3, random_state=42)
        embeddings_3d = reducer.fit_transform(embeddings_clean)
        
        # Creamos la visualización con UMAP
        viz_df = pd.DataFrame({
            'x': embeddings_3d[:, 0],
            'y': embeddings_3d[:, 1],
            'z': embeddings_3d[:, 2],
            'documento': [doc.split()[0] for doc in results['documents']],
            'documento_completo': results['documents'],
            'tipo': location_types
        })
        
        fig = px.scatter_3d(
            viz_df, 
            x='x', 
            y='y', 
            z='z',
            color='tipo',
            hover_data=['documento_completo'],
            text='documento',
            title='Visualización 3D de Embeddings con UMAP',
            labels={'x': 'UMAP 1', 'y': 'UMAP 2', 'z': 'UMAP 3'}
        )
        
        fig.update_traces(
            marker=dict(size=10, opacity=0.8), 
            selector=dict(mode='markers+text'),
            textposition='top center'
        )
        
        fig.update_layout(
            scene=dict(
                xaxis_title='UMAP 1',
                yaxis_title='UMAP 2',
                zaxis_title='UMAP 3'
            ),
            legend_title_text='Tipo de Ubicación',
            height=800,
            width=1000
        )
        
        fig.show()
        
    except ImportError:
        print("UMAP no está instalado. Usando t-SNE en 3D como alternativa.")
        
        # Si UMAP no está disponible, usar t-SNE en 3D
        tsne = TSNE(n_components=3, perplexity=5, random_state=42)
        embeddings_3d = tsne.fit_transform(embeddings_clean)
        
        viz_df = pd.DataFrame({
            'x': embeddings_3d[:, 0],
            'y': embeddings_3d[:, 1],
            'z': embeddings_3d[:, 2],
            'documento': [doc.split()[0] for doc in results['documents']],
            'documento_completo': results['documents'],
            'tipo': location_types
        })
        
        fig = px.scatter_3d(
            viz_df, 
            x='x', 
            y='y', 
            z='z',
            color='tipo',
            hover_data=['documento_completo'],
            text='documento',
            title='Visualización 3D de Embeddings con t-SNE',
            labels={'x': 't-SNE 1', 'y': 't-SNE 2', 'z': 't-SNE 3'}
        )
        
        fig.update_traces(
            marker=dict(size=10, opacity=0.8), 
            selector=dict(mode='markers+text'),
            textposition='top center'
        )
        
        fig.update_layout(
            scene=dict(
                xaxis_title='t-SNE 1',
                yaxis_title='t-SNE 2',
                zaxis_title='t-SNE 3'
            ),
            legend_title_text='Tipo de Ubicación',
            height=800,
            width=1000
        )
        
        fig.show()

Verificando datos: NaN: False, Inf: False
Valor mínimo: -0.12222213, Valor máximo: 0.2263778
Después de limpieza: NaN: False, Inf: False
Valor mínimo: -5.476346766702235, Valor máximo: 8.743462787306358
Varianza explicada por cada componente:
Componente 1: 9.78%
Componente 2: 8.73%
Componente 3: 8.22%
Total: 26.72%


## Interpretación del gráfico 3D y los errores de PCA

**¿Qué estamos viendo?**

El gráfico anterior muestra los documentos transformados a un espacio 3D mediante PCA. Cada punto representa un documento y su ubicación está determinada por los valores de las 3 componentes principales.

**¿Qué significan los porcentajes?**

Los porcentajes de varianza explicada nos indican cuánta información de los embeddings originales se conserva en cada dimensión del nuevo espacio reducido. Si la suma es cercana al 100%, la visualización captura bien la estructura de los datos originales.

**¿Por qué aparecen advertencias?**

Las advertencias como `divide by zero` y `overflow encountered` ocurren cuando los embeddings tienen:
- Valores extremadamente grandes o pequeños
- Estructuras muy dispersas (muchos ceros)
- Problemas de escala entre diferentes dimensiones

La normalización que aplicamos (StandardScaler) ayuda a reducir estos problemas.

Ahora veamos una visualización 2D con t-SNE que suele ser más intuitiva para visualizar clusters de documentos similares:

También podemos visualizar los embeddings en 2D, lo que puede facilitar la interpretación:

In [17]:
# Reducir a 2D con t-SNE para mejor visualización de clusters
tsne = TSNE(n_components=2, perplexity=5, random_state=42)
embeddings_2d = tsne.fit_transform(embeddings_array)

# Crear dataframe para visualización
viz_df_2d = pd.DataFrame({
    'x': embeddings_2d[:, 0],
    'y': embeddings_2d[:, 1],
    'documento': [doc.split()[0] for doc in results['documents']],  # Solo el nombre para etiquetas
    'documento_completo': results['documents'],
    'tipo': location_types
})

# Crear gráfico interactivo con hover
fig = px.scatter(
    viz_df_2d, 
    x='x', 
    y='y',
    color='tipo',
    hover_data=['documento_completo'],
    text='documento',
    title='Visualización de Embeddings con t-SNE',
    labels={'x': 't-SNE 1', 'y': 't-SNE 2'},
    template='plotly_white'
)

fig.update_traces(
    marker=dict(size=12, line=dict(width=1, color='DarkSlateGrey')),
    selector=dict(mode='markers+text')
)

fig.update_layout(
    height=700,
    width=1000,
    xaxis_title='t-SNE Dimensión 1',
    yaxis_title='t-SNE Dimensión 2',
    legend_title_text='Tipo de Ubicación'
)

fig.show()

## Similitud entre Documentos

Visualicemos la similitud del coseno entre los documentos para entender mejor cómo funciona la recuperación:

In [ ]:
# Función para calcular similitud del coseno entre dos vectores
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return dot_product / (norm_v1 * norm_v2)

# Calcular matriz de similitud entre todos los documentos
n_docs = len(embeddings_array)
similarity_matrix = np.zeros((n_docs, n_docs))

for i in range(n_docs):
    for j in range(n_docs):
        similarity_matrix[i, j] = cosine_similarity(embeddings_array[i], embeddings_array[j])

# Crear dataframe con nombres de documentos únicos
# Añadimos un índice para asegurar que cada nombre sea único
doc_names = [f"Doc_{i+1}" for i in range(n_docs)]  # Nombres genéricos completamente únicos
doc_labels = [doc.split()[0] for doc in results['documents']]  # Primeras palabras para referencia

# Crear el dataframe de similitud con los nombres únicos
similarity_df = pd.DataFrame(similarity_matrix, index=doc_names, columns=doc_names)

# Crear un diccionario que mapee los nombres generados a las etiquetas originales
name_to_label = {doc_name: f"{doc_label} ({doc_name})" for doc_name, doc_label in zip(doc_names, doc_labels)}

# Visualizar mapa de calor de similitud
fig = px.imshow(
    similarity_df,
    text_auto='.2f',
    aspect="auto",
    color_continuous_scale='RdBu_r',
    title='Matriz de Similitud del Coseno entre Documentos',
    labels={"x": "Documento", "y": "Documento", "color": "Similitud"}
)

# Actualizar las etiquetas de los ejes con los nombres originales
fig.update_layout(
    height=800,
    width=900,
    xaxis_title="Documento",
    yaxis_title="Documento",
)

# Para mejorar la legibilidad, añadir anotaciones para los documentos más importantes
# o eliminar comentar si hay demasiados documentos
"""
annotations = []
for i, doc_name in enumerate(doc_names):
    annotations.append(
        dict(
            x=i,
            y=-0.3,
            text=doc_labels[i],
            showarrow=False,
            xref="x",
            yref="paper",
            textangle=-45
        )
    )
fig.update_layout(annotations=annotations)
"""

fig.show()

# También podemos crear una visualización más interactiva usando Plotly Graph Objects
# Esta versión mostrará el contenido completo del documento al hacer hover
hover_text = [[f"Doc1: {results['documents'][i]}<br>Doc2: {results['documents'][j]}<br>Similitud: {similarity_matrix[i, j]:.2f}" 
              for j in range(n_docs)] for i in range(n_docs)]

fig2 = go.Figure(data=go.Heatmap(
    z=similarity_matrix,
    x=doc_names,
    y=doc_names,
    hoverongaps=False,
    hoverinfo="text",
    text=hover_text,
    colorscale='RdBu_r',
))

fig2.update_layout(
    title="Matriz de Similitud Interactiva (hover para ver detalles)",
    height=800,
    width=900,
    xaxis_title="Documento",
    yaxis_title="Documento",
)

fig2.show()

# También podemos crear una tabla que mapee los IDs de documentos a sus nombres
doc_mapping_df = pd.DataFrame({
    'ID': doc_names,
    'Nombre': [doc.split()[0] for doc in results['documents']],
    'Documento': results['documents'],
    'Tipo': location_types
})
display(doc_mapping_df)

## Demostración de Consulta

Finalmente, veamos cómo funciona una consulta real y qué documentos recupera:

In [15]:
# Función para probar consultas y visualizar resultados
def test_query(query_text, top_k=5):
    # Obtener embedding de la consulta
    query_embedding = embeddings.embed_query(query_text)
    
    # Calcular similitud con todos los documentos
    similarities = []
    for emb in embeddings_array:
        similarity = cosine_similarity(query_embedding, emb)
        similarities.append(similarity)
    
    # Crear dataframe con resultados
    results_df = pd.DataFrame({
        'documento': results['documents'],
        'similitud': similarities,
        'tipo': location_types
    })
    
    # Ordenar por similitud
    results_df = results_df.sort_values('similitud', ascending=False).reset_index(drop=True)
    
    # Crear gráfico de barras para los top_k resultados
    top_results = results_df.head(top_k)
    
    fig = px.bar(
        top_results, 
        y='documento', 
        x='similitud',
        color='tipo',
        orientation='h',
        title=f'Top {top_k} documentos más similares a: "{query_text}"',
        labels={'similitud': 'Similitud del coseno', 'documento': 'Documento'},
        template='plotly_white'
    )
    
    fig.update_layout(
        yaxis={'categoryorder':'total ascending'},
        xaxis_title="Similitud del coseno",
        yaxis_title="Documento"
    )
    
    fig.show()
    
    return top_results

# Probar con una consulta sobre Robledo
query = "¿Cuál es la comida favorita en Robledo?"
print(f"Consulta: '{query}'")
test_query(query)

Consulta: '¿Cuál es la comida favorita en Robledo?'


,documento,similitud,tipo
0,Robledo 13.5 150000 Empanadas Cerro El Volador...,0.692416,Commune
1,Bello 142.36 532154 Arepa de Chocolo Puerta de...,0.620638,Municipality
2,Castilla 6.2 110000 Frijoles con Chicharrón Pa...,0.614378,Commune
3,Manrique 7.5 120000 Chorizo Antioqueño Parque ...,0.605474,Commune
4,El Poblado 23.1 130000 Sancocho Antioqueño Par...,0.602082,Commune


## Conclusiones

En este notebook hemos visto:

1. Cómo acceder a los documentos almacenados en ChromaDB
2. La estructura y dimensionalidad de los embeddings
3. Cómo visualizar embeddings de alta dimensionalidad utilizando PCA y t-SNE
4. La similitud semántica entre documentos a través de una matriz de similitud del coseno
5. Cómo funciona una consulta real, mostrando qué documentos son más similares

Esta visualización nos ayuda a entender mejor cómo el sistema RAG selecciona los documentos más relevantes para responder a las preguntas del usuario.